<a href="https://colab.research.google.com/github/syh0397/Charter-Dashboard-for-Newbies/blob/main/%EB%A7%A4%EB%AC%BC%EB%B3%84_%EA%B1%B0%EB%A6%AC_%EB%82%B4_%EC%A3%BC%EB%B3%80%EC%8B%9C%EC%84%A4_%EA%B0%AF%EC%88%98%EA%B5%AC%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### IMPORT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [144]:
import json
import requests
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/DATA/비저블 맵 프로젝트/Seoulhousetrans.csv')
df.reset_index()
df

,Unnamed: 0,자치구명,법정동코드,법정동명,지번주소,건물명,층,임대건물명,임대면적,전월세구분,보증금,임대료,건축년도,연식,latitude,longitude
0,1149,영등포구,13000,양평동6가,서울특별시 영등포구 양평동6가 93-2,선유도디와이파크,4,오피스텔,19.52,준전세,11000,40,2017.0,5,37.543427,126.888418
1,1302,영등포구,13000,양평동6가,서울특별시 영등포구 양평동6가 93-2,선유도디와이파크,6,오피스텔,18.92,전세,13125,0,2017.0,5,37.543427,126.888418
2,1303,영등포구,13000,양평동6가,서울특별시 영등포구 양평동6가 93-2,선유도디와이파크,5,오피스텔,18.92,전세,12600,0,2017.0,5,37.543427,126.888418
3,1304,영등포구,13000,양평동6가,서울특별시 영등포구 양평동6가 93-2,선유도디와이파크,3,오피스텔,18.92,전세,9923,0,2017.0,5,37.543427,126.888418
4,1109,영등포구,13000,양평동6가,서울특별시 영등포구 양평동6가 84,유성빌라(84-0),2,다세대/연립,50.82,준월세,1000,50,1986.0,36,37.543388,126.889033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1186,407,서초구,10300,우면동,서울특별시 서초구 우면동 765,미라쉘,3,오피스텔,19.40,준월세,3000,45,2015.0,7,37.456478,127.015923
1187,112,서초구,10300,우면동,서울특별시 서초구 우면동 764,계룡리슈빌S,7,아파트,21.59,준월세,1000,70,2014.0,8,37.456461,127.016237
1188,411,서초구,10300,우면동,서울특별시 서초구 우면동 764,계룡리슈빌S,7,아파트,21.59,준월세,10000,45,2014.0,8,37.456461,127.016237
1189,662,서초구,11100,신원동,서울특별시 서초구 신원동 619,서초포레스타6단지,4,아파트,49.34,전세,13067,0,2015.0,7,37.449935,127.055830


### API CONTROL

In [145]:
class KakaoLocalAPI:
    """
    Kakao Local API 컨트롤러
    """

    def __init__(self, rest_api_key):
        """
        Rest API키 초기화 및 기능 별 URL 설정
        """

        # REST API 키 설정
        self.rest_api_key = '650c8233432728848e448a6d76f3f155'
        self.headers = {"Authorization": "KakaoAK {}".format(rest_api_key)}

        # 서비스 별 URL 설정

        # 01 주소 검색
        self.URL_01 = "https://dapi.kakao.com/v2/local/search/address.json"
        # 02 좌표-행정구역정보 변환
        self.URL_02 = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
        # 03 좌표-주소 변환
        self.URL_03 = "https://dapi.kakao.com/v2/local/geo/coord2address.json"
        # 04 좌표계 변환
        self.URL_04 = "https://dapi.kakao.com/v2/local/geo/transcoord.json"
        # 05 키워드 검색
        self.URL_05 = "https://dapi.kakao.com/v2/local/search/keyword.json"
        # 06 카테고리 검색
        self.URL_06 = "https://dapi.kakao.com/v2/local/search/category.json"

    def search_address(self, query, analyze_type=None, page=None, size=None):
        """
        01 주소 검색
        """
        params = {"query": f"{query}"}

        if analyze_type != None:
            params["analyze_type"] = f"{analyze_type}"

        if page != None:
            params['page'] = f"{page}"

        if size != None:
            params['size'] = f"{size}"

        res = requests.get(self.URL_01, headers=self.headers, params=params)
        document = json.loads(res.text)

        return document
    
    def geo_coord2regioncode(self, x, y, input_coord=None, output_coord=None):
        """
        02 좌표-행정구역정보 변환
        """
        params = {"x": f"{x}",
                  "y": f"{y}"}
        
        if input_coord != None:
            params['input_coord'] = f"{input_coord}"
        
        if output_coord != None:
            params['output_coord'] = f"{output_coord}"
            
        res = requests.get(self.URL_02, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document
        
    
    def geo_coord2address(self, x, y, input_coord=None):
        """
        03 좌표-주소 변환
        """
        params = {"x": f"{x}",
                  "y": f"{y}"}
        
        if input_coord != None:
            params['input_coord'] = f"{input_coord}"
            
        res = requests.get(self.URL_03, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document
        
    
    def geo_transcoord(self, x, y, output_coord, input_coord=None):
        """
        04 좌표계 변환
        """
        params = {"x": f"{x}",
                  "y": f"{y}",
                  "output_coord": f"{output_coord}"}
        
        if input_coord != None:
            params['input_coord'] = f"{input_coord}"
        
        res = requests.get(self.URL_04, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document
        
    
    def search_keyword(self,query,category_group_code=None,x=None,y=None,radius=None,rect=None,page=None,size=None,sort=None):
        """
        05 키워드 검색
        """
        params = {"query": f"{query}"}
        
        if category_group_code != None:
            params['category_group_code'] = f"{category_group_code}"
        if x != None:
            params['x'] = f"{x}"
        if y != None:
            params['y'] = f"{y}"
        if radius != None:
            params['radius'] = f"{radius}"
        if rect != None:
            params['rect'] = f"{rect}"
        if page != None:
            params['page'] = f"{page}"
        if size != None:
            params['size'] = f"{params}"
        if sort != None:
            params['sort'] = f"{sort}"
        
        res = requests.get(self.URL_05, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document
    
            
        
    def search_category(self, category_group_code, x, y, radius=None, rect=None, page=None, size=None, sort=None):
        """
        06 카테고리 검색
        """
        params = {'category_group_code': f"{category_group_code}",
                  'x': f"{x}",
                  'y': f"{y}"}
        
        if radius != None:
            params['radius'] = f"{radius}"
        if rect != None:
            params['rect'] = f"{rect}"
        if page != None:
            params['page'] = f"{page}"
        if size != None:
            params['size'] = f"{size}"
        if sort != None:
            params['sort'] = f"{sort}"
            
        res = requests.get(self.URL_06, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document

### API KEY AND TEST

In [186]:
# REST API 키
rest_api_key = "650c8233432728848e448a6d76f3f155"

kakao = KakaoLocalAPI(rest_api_key)

In [148]:
dong = df['지번주소'][0:10]
address = set(dong)
address

{'서울특별시 영등포구 양평동4가 165',
 '서울특별시 영등포구 양평동5가 22-1',
 '서울특별시 영등포구 양평동5가 51',
 '서울특별시 영등포구 양평동6가 2-3',
 '서울특별시 영등포구 양평동6가 84',
 '서울특별시 영등포구 양평동6가 93-2'}

In [214]:
range(len(df))

range(0, 1191)

### 편의점 300미터 안

In [217]:
in_conv_total_300 = []
office = []

for i in range(len(df)):
  # 편의점
  category_group_code = "CS2"
  x = df['longitude'][i]
  y = df['latitude'][i]
  radius = 300

  ## Request
  result_ = kakao.search_category(category_group_code, x, y, radius)
  places = requests.get(url, params=params, headers=headers).json()['documents']
  
  Conv_total_count = result_['meta']['total_count']
  # 지번주소
  office.append(df['지번주소'][i])
  # 편의점 갯수
  in_conv_total_300.append(Conv_total_count)

# #데이터 프레임
num = pd.DataFrame()
# convin['orgin_add'] = orgin_add
num['office'] = office
num['in_conv_total_300'] = in_conv_total_300

In [226]:
num.to_csv('/content/drive/MyDrive/DATA/비저블 맵 프로젝트/편의점300미터.csv')

In [227]:
df['conv_num'] = in_conv_total_300

In [229]:
no_cov = df[df['conv_num'] == 0]

,Unnamed: 0,자치구명,법정동코드,법정동명,지번주소,건물명,층,임대건물명,임대면적,전월세구분,보증금,임대료,건축년도,연식,latitude,longitude,conv_num
1069,1107,강남구,10300,개포동,서울특별시 강남구 개포동 121-2,원아트빌라,1,다세대/연립,35.10,준월세,4000,50,1995.0,27,37.476465,127.067260,0
1073,1175,강남구,10300,개포동,서울특별시 강남구 개포동 121-6,명성빌라,1,다세대/연립,44.98,준전세,12000,31,1993.0,29,37.476041,127.068264,0
1180,438,서초구,10900,내곡동,서울특별시 서초구 내곡동,서초포레스타2단지,14,아파트,49.33,준월세,1946,42,2015.0,7,37.458246,127.076917,0


### 음식점 300미터 안

In [254]:
in_din_total_300 = []
office = []

for i in range(len(df)):
  # 음식점 
  category_group_code = "FD6"
  x = df['longitude'][i]
  y = df['latitude'][i]
  radius = 300

  ## Request
  result_ = kakao.search_category(category_group_code, x, y, radius)
  # places = requests.get(url, params=params, headers=headers).json()['documents']
  
  Dini_total_count = result_['meta']['total_count']
  # 지번주소
  # office.append(df['지번주소'][i])
  # 편의점 갯수
  in_din_total_300.append(Dini_total_count)

# #데이터 프레임
num = pd.DataFrame()
# convin['orgin_add'] = orgin_add
# num['office'] = office
num['in_din_total_300'] = in_din_total_300

In [259]:
df['in_din_total_300'] = in_din_total_300

### 마트 1000미터 안

In [261]:
in_mart_total_1000 = []
office = []

for i in range(len(df)):
  # 마트
  category_group_code = "MT1"
  x = df['longitude'][i]
  y = df['latitude'][i]
  radius = 1000

  ## Request
  result_ = kakao.search_category(category_group_code, x, y, radius)
  # places = requests.get(url, params=params, headers=headers).json()['documents']
  
  mart_total_count = result_['meta']['total_count']
  # 지번주소
  # office.append(df['지번주소'][i])
  # 편의점 갯수
  in_mart_total_1000.append(mart_total_count)

# #데이터 프레임
num = pd.DataFrame()
# convin['orgin_add'] = orgin_add
# num['office'] = office
num['in_mart_total_1000'] = in_mart_total_1000

In [ ]:
df['in_mart_total_1000'] = in_mart_total_1000

### 지하철 500미터 안

In [264]:
in_subway_total_500 = []
office = []

for i in range(len(df)):
  # 지하철
  category_group_code = "SW8"
  x = df['longitude'][i]
  y = df['latitude'][i]
  radius = 500

  ## Request
  result_ = kakao.search_category(category_group_code, x, y, radius)
  # places = requests.get(url, params=params, headers=headers).json()['documents']
  
  sub_total_count = result_['meta']['total_count']
  # 지번주소
  # office.append(df['지번주소'][i])
  # 편의점 갯수
  in_subway_total_500.append(sub_total_count)

# #데이터 프레임
num = pd.DataFrame()
# convin['orgin_add'] = orgin_add
# num['office'] = office
num['in_subway_total_500'] = in_subway_total_500

In [ ]:
df['in_subway_total_500'] = in_subway_total_500
df

### 병원 1000미터 안

In [266]:
in_hosp_total_1000 = []
# office = []

for i in range(len(df)):
  # 병원
  category_group_code = "HP8"
  x = df['longitude'][i]
  y = df['latitude'][i]
  radius = 1000

  ## Request
  result_ = kakao.search_category(category_group_code, x, y, radius)
  # places = requests.get(url, params=params, headers=headers).json()['documents']
  
  hos_total_count = result_['meta']['total_count']
  # 지번주소
  # office.append(df['지번주소'][i])
  # 편의점 갯수
  in_hosp_total_1000.append(hos_total_count)

# #데이터 프레임
num = pd.DataFrame()
# convin['orgin_add'] = orgin_add
# num['office'] = office
num['in_hosp_total_1000'] = in_hosp_total_1000

In [ ]:
df['in_hosp_total_1000'] = in_hosp_total_1000
df.head()

### 카페 300미터안

In [268]:
in_cafe_total_300 = []
# office = []

for i in range(len(df)):
  # cafe
  category_group_code = "CE7"
  x = df['longitude'][i]
  y = df['latitude'][i]
  radius = 300

  # Request
  result_ = kakao.search_category(category_group_code, x, y, radius)
  # places = requests.get(url, params=params, headers=headers).json()['documents']
  
  cafe_total_count = result_['meta']['total_count']
  # 지번주소
  # office.append(df['지번주소'][i])
  # 편의점 갯수
  in_cafe_total_300.append(cafe_total_count)

# DataFrame
num = pd.DataFrame()
# convin['orgin_add'] = orgin_add
# num['office'] = office
num['in_cafe_total_300'] = in_cafe_total_300
df['in_cafe_total_300'] = in_cafe_total_300

### 경찰서 500미터안

In [289]:
police_500 = []

for i in range(len(df)):
  x = df['longitude'][i]
  y = df['latitude'][i]
  ## 검색어 지정
  query = "경찰서"
  ## request
  result_keyword = kakao.search_keyword(query,
                                x = x,
                                y = y,
                                radius = 500,
                                sort = 'distance')
  police_total_count = result_keyword['meta']['total_count']

  police_500.append(police_total_count)
  num = pd.DataFrame()
  num['police_500'] = police_500

In [290]:
num

,police_500
0,1
1,1
2,1
3,1
4,1
...,...
1186,1
1187,1
1188,1
1189,0


In [291]:
df['police_500'] = police_500

### 동사무소  500

In [299]:
center_500 = []

for i in range(len(df)):
  x = df['longitude'][i]
  y = df['latitude'][i]
  ## 검색어 지정
  query = "동사무소"
  ## request
  result_keyword = kakao.search_keyword(query,
                                x = x,
                                y = y,
                                radius = 500,
                                sort = 'distance')
  center_total_count = result_keyword['meta']['total_count']

  center_500.append(center_total_count)
  num = pd.DataFrame()
  num['center_500'] = center_500

In [301]:
df['center_500'] = center_500
df

,Unnamed: 0,자치구명,법정동코드,법정동명,지번주소,건물명,층,임대건물명,임대면적,전월세구분,...,latitude,longitude,conv_num,in_din_total_300,in_mart_total_1000,in_subway_total_500,in_hosp_total_1000,in_cafe_total_300,police_500,center_500
0,1149,영등포구,13000,양평동6가,서울특별시 영등포구 양평동6가 93-2,선유도디와이파크,4,오피스텔,19.52,준전세,...,37.543427,126.888418,3,10,2,1,29,4,1,1
1,1302,영등포구,13000,양평동6가,서울특별시 영등포구 양평동6가 93-2,선유도디와이파크,6,오피스텔,18.92,전세,...,37.543427,126.888418,3,10,2,1,29,4,1,1
2,1303,영등포구,13000,양평동6가,서울특별시 영등포구 양평동6가 93-2,선유도디와이파크,5,오피스텔,18.92,전세,...,37.543427,126.888418,3,10,2,1,29,4,1,1
3,1304,영등포구,13000,양평동6가,서울특별시 영등포구 양평동6가 93-2,선유도디와이파크,3,오피스텔,18.92,전세,...,37.543427,126.888418,3,10,2,1,29,4,1,1
4,1109,영등포구,13000,양평동6가,서울특별시 영등포구 양평동6가 84,유성빌라(84-0),2,다세대/연립,50.82,준월세,...,37.543388,126.889033,3,11,2,0,28,6,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1186,407,서초구,10300,우면동,서울특별시 서초구 우면동 765,미라쉘,3,오피스텔,19.40,준월세,...,37.456478,127.015923,2,8,1,0,6,3,1,0
1187,112,서초구,10300,우면동,서울특별시 서초구 우면동 764,계룡리슈빌S,7,아파트,21.59,준월세,...,37.456461,127.016237,2,8,1,0,6,3,1,0
1188,411,서초구,10300,우면동,서울특별시 서초구 우면동 764,계룡리슈빌S,7,아파트,21.59,준월세,...,37.456461,127.016237,2,8,1,0,6,3,1,0
1189,662,서초구,11100,신원동,서울특별시 서초구 신원동 619,서초포레스타6단지,4,아파트,49.34,전세,...,37.449935,127.055830,6,16,2,1,14,5,0,2


In [302]:
df.to_csv('/content/drive/MyDrive/DATA/비저블 맵 프로젝트/서울시전월세_점수추가.csv')

### 끝